## Goals this lesson and exercise set:
 - Understand how to interact with a programmatic PI for the sake of acquring data from web-based sources
 - In this case specifically, we want to obtain all of the information at the store data API that we began to examine in the lesson

### How we are going to go about doing that:
 - Interact and navigate with the API further
 - Utilize the requests library for interacting with the web pythonically
 - Cement the process of working with a looped structure to navigate and corroborate paginated data

In [1]:
# basic imports
import pandas as pd
import requests

In [2]:
url = 'https://python.zgulde.net/api/v1'

In [3]:
# how do i interact with that? requests.get()
response = requests.get(url)

In [4]:
# response itself holds the HTTP response
response

<Response [200]>

In [5]:
# to grab the content of the response, use .text or
# .json() for RESTful API information that follows
# the structure of what we anticipate

In [6]:
# if we use that response.json():
response.json()

{'payload': {'routes': ['/stores',
   '/stores/{store_id}',
   '/items',
   '/items/{item_id}',
   '/sales',
   '/sales/{sale_id}']},
 'status': 'ok'}

In [7]:
store_url = 'https://python.zgulde.net/api/v1/stores'

In [8]:
# put the contents of store_response into a variable
store_response = requests.get(store_url).json()

In [9]:
# inside of store_response i have a payload and a status
# inside of the payload, I have store content as well as 
# information about the pagination of the information
# I only have one page here, so let's store it as a
# dataframe
store_response['payload']['stores'][0:3]

[{'store_address': '12125 Alamo Ranch Pkwy',
  'store_city': 'San Antonio',
  'store_id': 1,
  'store_state': 'TX',
  'store_zipcode': '78253'},
 {'store_address': '9255 FM 471 West',
  'store_city': 'San Antonio',
  'store_id': 2,
  'store_state': 'TX',
  'store_zipcode': '78251'},
 {'store_address': '2118 Fredericksburg Rdj',
  'store_city': 'San Antonio',
  'store_id': 3,
  'store_state': 'TX',
  'store_zipcode': '78201'}]

In [10]:
store_df = store_response['payload']['stores']

In [11]:
store_url

'https://python.zgulde.net/api/v1/stores'

In [12]:
# if i wanted to grab a new endpoint from the initial payload I 
# could reference it as thus:
requests.get(
    'https://python.zgulde.net/api/v1').json(
)['payload']['routes'][2]

'/items'

In [13]:
# going forward, I need to grab items and sales

In [14]:
item_url = 'https://python.zgulde.net/api/v1/items'
sales_url = 'https://python.zgulde.net/api/v1/sales'

In [15]:
# let's check out items:
# looks like items actually has more than one page.
# let's examine how to naviage that!
requests.get(item_url).json()['payload']['next_page']

'/api/v1/items?page=2'

In [16]:
# api/v1/items?page=2 represents a new endpoint.
# how could we change that in the structure of a loop?

In [17]:
item_url

'https://python.zgulde.net/api/v1/items'

In [18]:
requests.get(item_url).json()['payload']['max_page']

3

In [19]:
# I think this is the structure, let's confirm:
type(requests.get(
    item_url+'?page=3').json(
)['payload']['next_page'])

NoneType

Takeaways here: the max page contains a key for the next page always, including on the max page, but when on the max page it leads to a Nonetype

In [20]:
# two ways of corroborating this info:

Way 1: create a while loop that runs until it its a next page key of None

In [21]:
# requests.get(base_url+endpoint).json()

In [22]:
# let's get items with a while loop:
# create a new base url for stores:
# let's create a new base url and endpoint,
# change the endpoint programmatically in the loop
base_url = 'https://python.zgulde.net'
endpoint = '/api/v1/items'
# initialize an empty list
items = []

# make a while loop that does the thing 
# until its no longer true:

while True:
    url = base_url + endpoint
    response = requests.get(url)
    data = response.json()
    # print(f'Getting page {data['payload']['page']} of {data['payload']['max_page']}: {url}', 
    #       end='')
    items.extend(data['payload']['items'])
    endpoint = data['payload']['next_page']
    if endpoint is None:
        break

In [23]:
items_df = pd.DataFrame(items)

In [24]:
store_df = pd.DataFrame(store_df)

In [25]:
items_df.columns

Index(['item_brand', 'item_id', 'item_name', 'item_price', 'item_upc12',
       'item_upc14'],
      dtype='object')

In [26]:
store_df.columns

Index(['store_address', 'store_city', 'store_id', 'store_state',
       'store_zipcode'],
      dtype='object')

In [27]:
store_df.head()

,store_address,store_city,store_id,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,9255 FM 471 West,San Antonio,2,TX,78251
2,2118 Fredericksburg Rdj,San Antonio,3,TX,78201
3,516 S Flores St,San Antonio,4,TX,78204
4,1520 Austin Hwy,San Antonio,5,TX,78218


Way 2: I have a max page key immediately in my payload -- I can create a for loop that runs through the information with a range

In [28]:
base_url

'https://python.zgulde.net'

In [29]:
# let's examine sales for a proper endpoint:
sales_url

'https://python.zgulde.net/api/v1/sales'

In [30]:
requests.get(sales_url).json()['payload'].keys()

dict_keys(['max_page', 'next_page', 'page', 'previous_page', 'sales'])

In [31]:
requests.get(sales_url).json()['payload']['max_page']

183

In [32]:
requests.get(sales_url).json()['payload']['next_page']

'/api/v1/sales?page=2'

In [33]:
base_url = 'https://python.zgulde.net/api/v1/sales?page='
endpoint = '1'

In [34]:
requests.get(base_url+'183').json()['payload']['sales'][:3]

[{'item': 50,
  'sale_amount': 77.0,
  'sale_date': 'Wed, 15 Oct 2014 00:00:00 GMT',
  'sale_id': 910001,
  'store': 9},
 {'item': 50,
  'sale_amount': 52.0,
  'sale_date': 'Thu, 16 Oct 2014 00:00:00 GMT',
  'sale_id': 910002,
  'store': 9},
 {'item': 50,
  'sale_amount': 65.0,
  'sale_date': 'Fri, 17 Oct 2014 00:00:00 GMT',
  'sale_id': 910003,
  'store': 9}]

In [35]:
for n in range(1,5):
    print(n)

1
2
3
4


In [36]:
# approach #2 with sales:
# make a for loop, only change the number based on
# what we get in the max page:
# base_url = 'https://python.zgulde.net/api/v1/sales?page='
# sales = []
# for page_no in range(1,184):
#     endpoint = str(page_no)
#     response = requests.get(
#         base_url+endpoint).json()['payload']['sales']
#     print(f'Getting page {endpoint} of 183', end='')
#     sales.extend(response)

In [ ]:
# sales_df = pd.Dataframe(sales)

In [37]:
# save a little time for the review by reading in a cached
sales = pd.read_csv('sales.csv')

In [38]:
sales.head()

,item,sale_amount,sale_date,sale_id,store
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1


In [39]:
sales.columns

Index(['item', 'sale_amount', 'sale_date', 'sale_id', 'store'], dtype='object')

In [43]:
items_df.shape

(50, 6)

In [46]:
sales.head(2)

,item,sale_amount,sale_date,sale_id,store
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1


In [45]:
items_df.head(2)

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925


In [41]:
sales.item.nunique()

50

In [51]:
sales.store.nunique() == store_df.store_id.nunique()

True

In [48]:
store_df.shape

(10, 5)

In [52]:
sales = sales.rename(columns={'item': 'item_id', 'store': 'store_id'})

In [53]:
# sanity check for my rename:
sales.columns

Index(['item_id', 'sale_amount', 'sale_date', 'sale_id', 'store_id'], dtype='object')

In [54]:
sales.shape

(913000, 5)

In [57]:
type(sales)

pandas.core.frame.DataFrame

In [58]:
type(items_df)

pandas.core.frame.DataFrame

In [ ]:
# use more consistent names than i did probably -_-

In [59]:
df = pd.merge(sales, items_df, how='left', on='item_id')

In [62]:
df.shape

(913000, 10)

In [63]:
type(store_df)

pandas.core.frame.DataFrame

In [64]:
df = pd.merge(df, store_df, how='left', on='store_id')

In [65]:
df.columns

Index(['item_id', 'sale_amount', 'sale_date', 'sale_id', 'store_id',
       'item_brand', 'item_name', 'item_price', 'item_upc12', 'item_upc14',
       'store_address', 'store_city', 'store_state', 'store_zipcode'],
      dtype='object')

In [66]:
df.head()

,item_id,sale_amount,sale_date,sale_id,store_id,item_brand,item_name,item_price,item_upc12,item_upc14,store_address,store_city,store_state,store_zipcode
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
